# 📘 Exercício II - Análise de Propensão de Geração de Crédito em Notas Fiscais

## 📌 Etapa 1: Criação de Variáveis Temporais

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Carregar base
df = pd.read_pickle("base_nfp.pkl")
df["Data Emissão"] = pd.to_datetime(df["Data Emissão"])

# Criar variáveis temporais
df["dia_semana"] = df["Data Emissão"].dt.weekday
df["fim_de_semana"] = df["dia_semana"].isin([5, 6]).astype(int)
df["dia_mes"] = df["Data Emissão"].dt.day
df["mes"] = df["Data Emissão"].dt.month
df["trimestre"] = df["Data Emissão"].dt.quarter


## 📌 Etapa 2: Categorias por Quantis do Valor da Nota

In [ ]:

# Criar variáveis categóricas por quantis
for q in [5, 10, 20, 50]:
    df[f"valor_nf_q{q}"] = pd.qcut(df["Valor NF"], q, duplicates="drop").astype(str)


## 📌 Etapa 3: Cálculo de Information Value (IV)

In [ ]:

# Flag de retorno
df["flag_retorno"] = (df["Retorno"] > 0).astype(int)

def calculate_woe_iv_auto(df, feature, target):
    eps = 0.0001
    df_temp = df[[feature, target]].copy()
    df_temp = df_temp[df_temp[feature].notna()]
    grouped = pd.crosstab(df_temp[feature], df_temp[target])
    if 0 not in grouped.columns or 1 not in grouped.columns:
        return None
    grouped.columns = ["No_Event", "Event"]
    grouped["Dist_Event"] = grouped["Event"] / grouped["Event"].sum()
    grouped["Dist_No_Event"] = grouped["No_Event"] / grouped["No_Event"].sum()
    grouped["WoE"] = np.log((grouped["Dist_Event"] + eps) / (grouped["Dist_No_Event"] + eps))
    grouped["IV"] = (grouped["Dist_Event"] - grouped["Dist_No_Event"]) * grouped["WoE"]
    return grouped["IV"].sum()

variaveis = ["dia_semana", "fim_de_semana", "dia_mes", "mes", "trimestre",
             "valor_nf_q5", "valor_nf_q10", "valor_nf_q20", "valor_nf_q50"]

iv_scores = {v: calculate_woe_iv_auto(df, v, "flag_retorno") for v in variaveis}
iv_scores = dict(sorted(iv_scores.items(), key=lambda item: item[1], reverse=True))
iv_scores


## 📌 Etapa 4: Análise Descritiva no Tempo

In [ ]:

# Agregar por trimestre
df["periodo_trimestre"] = df["Data Emissão"].dt.to_period("Q").astype(str)
notas_por_trimestre = df.groupby("periodo_trimestre").size()
notas_por_categoria = df.groupby(["periodo_trimestre", "categoria"]).size().unstack(fill_value=0)
retorno_por_trimestre = df.groupby("periodo_trimestre")["flag_retorno"].mean()

# Plots
fig, axs = plt.subplots(3, 1, figsize=(14, 15), sharex=True)
axs[0].plot(notas_por_trimestre.index, notas_por_trimestre.values, marker="o")
axs[0].set_title("Quantidade de Notas por Trimestre")
axs[0].grid(True)

notas_por_categoria.plot(kind="bar", stacked=True, ax=axs[1], colormap="tab20")
axs[1].set_title("Distribuição de Notas por Categoria (Trimestral)")
axs[1].legend(title="Categoria", bbox_to_anchor=(1.05, 1))
axs[1].grid(True)

axs[2].plot(retorno_por_trimestre.index, retorno_por_trimestre.values, marker="s", color="green")
axs[2].set_title("Proporção de Notas com Retorno (> 0) por Trimestre")
axs[2].grid(True)
plt.tight_layout()
plt.show()



## ✅ Conclusão

- As variáveis que apresentaram maior IV foram as categorias criadas a partir de `Valor NF`, principalmente `valor_nf_q50` (IV ≈ 0.15), o que indica **poder preditivo moderado**.
- Variáveis temporais como `dia da semana`, `mês` e `trimestre` apresentaram IVs fracos ou muito fracos.
- Aumentar o número de categorias em `Valor NF` aumentou o IV, indicando maior poder de discriminação, mas esse efeito pode ser limitado por risco de overfitting.
- A análise gráfica mostrou variações sazonais e operacionais importantes nos dados de emissão e retorno de crédito, o que pode indicar necessidade de **modelagem por segmento ou tempo**.

**Próximos passos recomendados:**  
- Aplicar WoE em variáveis contínuas discretizadas;  
- Testar modelos com e sem WoE para comparar performance;  
- Estudar estabilidade das variáveis ao longo do tempo (PSI);  
- Considerar variáveis externas (ex: sazonalidade, pandemia).  
